In [21]:
#!/.conda/envs/learn python
# -*- coding: utf-8 -*-

"""
多类别精度评定

预测结果和真值标签均为栅格
文件名相同，尺寸相同，不需要地理坐标系
~~~~~~~~~~~~~~~~
code by wHy
Aerospace Information Research Institute, Chinese Academy of Sciences
wanghaoyu191@mails.ucas.ac.cn
"""

'\n多类别精度评定\n\n预测结果和真值标签均为栅格\n文件名相同，尺寸相同，不需要地理坐标系\n~~~~~~~~~~~~~~~~\ncode by wHy\nAerospace Information Research Institute, Chinese Academy of Sciences\nwanghaoyu191@mails.ucas.ac.cn\n'

In [22]:
import os
from statistics import mean
import sys
import fnmatch
import numpy as np
import gdal
import ogr
import osr
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [23]:
os.environ['GDAL_DATA'] = r'C:\Users\75198\.conda\envs\learn\Lib\site-packages\GDAL-2.4.1-py3.6-win-amd64.egg-info\gata-data' #防止报error4错误

ground_truth_path = r'E:\xinjiang_huyang_hongliu\Huyang_test_0808\1-raster_label\1-raster_label_haze_lv3' # 存储真值标签的文件夹 真值标签应为栅格 带有地理坐标系 _for_clear_Evaluation
predict_path = r'C:\Users\75198\OneDrive\论文\SCI-3-3 Remote sensing data augmentation\240408补充实验\其它\LV3\clahe\predict_result' # 存储预测栅格的文件 带有地理坐标系

'''为了省事自动检索预测结果文件夹下的tif影像'''
pre_pic_name = fnmatch.filter(os.listdir(predict_path), '*.tif')
pre_pic_name_full = predict_path + '/' + pre_pic_name[0][:-4] + '.tif'

im_data_pre_full = []
im_data_true_full = []
listpic = fnmatch.filter(os.listdir(ground_truth_path), '*.tif')
for gt_name in listpic:
    gt_full_path = ground_truth_path + '/' + gt_name[:-4] + '.tif'
    pre_pic_name_full = predict_path + '/' + gt_name[:-4] + '_result.tif'
    '''以真值标签为参考，从预测栅格文件中裁剪出待评定部分'''
    # 获取真值标签地理坐标信息
    input_gt = gdal.Open(gt_full_path)

    # 获取预测结果数据
    input_pre = gdal.Open(pre_pic_name_full)

    # 定义裁剪窗口大小
    win_size = input_gt.RasterXSize

    # 整理数据
    im_data_pre = input_pre.ReadAsArray(0, 0, win_size, win_size)  # 读取预测结果对应区域的数据
    im_data_true = input_gt.ReadAsArray(0, 0, win_size, win_size) # 读取真值标签区域数据

    # print(im_data_pre.shape, im_data_true.shape)
    # print(type(im_data_pre))

    im_data_pre = list(im_data_pre.reshape(-1)) # 展平为一维
    im_data_true = list(im_data_true.reshape(-1)) # 展平为一维

    im_data_pre_full = im_data_pre_full + im_data_pre # 链接
    im_data_true_full = im_data_true_full + im_data_true

# print(np.shape(im_data_pre_full), np.shape(im_data_true_full))
    
unique_pre, count_pre=np.unique(im_data_pre_full,return_counts=True)
data_count_pre=dict(zip(unique_pre, count_pre))
unique_true,count_true=np.unique(im_data_true_full,return_counts=True)
data_count_true=dict(zip(unique_true,count_true))
# print(data_count_pre)
# print(data_count_true)

'''精度评定部分'''
'''计算混淆矩阵'''
cm = confusion_matrix(im_data_true_full, im_data_pre_full, normalize='true')
print(np.shape(cm))





(256, 256) (256, 256)
<class 'numpy.ndarray'>
(256, 256) (256, 256)
<class 'numpy.ndarray'>
(256, 256) (256, 256)
<class 'numpy.ndarray'>
(196608,) (196608,)
{0: 193618, 1: 88, 2: 2902}
{0: 188800, 1: 5499, 2: 2309}


(3, 3)
Confusion matrix:
[[9.97372881e-01 1.58898305e-05 2.61122881e-03]
 [6.92489544e-01 1.54573559e-02 2.92053101e-01]
 [6.52230403e-01 0.00000000e+00 3.47769597e-01]] 

Accuracy: 0.99383544921875
Balanced Accuracy: 0.7255670010831796
My Balanced Accuracy: 0.4535332781556858
Precision: 0.960001525573634
Recall: 0.7255670010831796
F1 Score: 0.8019486251106629




ValueError: Number of classes, 2, does not match size of target_names, 3. Try specifying the labels parameter